<a href="https://colab.research.google.com/github/manmeet3/Deep_Learning2/blob/master/Asg4/meta-learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Meta Learning
Using Omni glot dataset

In [ ]:
# ref: https://github.com/shashankhalo7/Omniglot_meta_learning/blob/master/osl.ipynb

In [2]:
!wget https://github.com/manmeet3/Deep_Learning2/raw/master/Asg4/images_background.zip

--2020-10-15 21:06:15--  https://github.com/manmeet3/Deep_Learning2/raw/master/Asg4/images_background.zip
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/manmeet3/Deep_Learning2/master/Asg4/images_background.zip [following]
--2020-10-15 21:06:16--  https://raw.githubusercontent.com/manmeet3/Deep_Learning2/master/Asg4/images_background.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9464212 (9.0M) [application/zip]
Saving to: ‘images_background.zip’

images_background.z 100%[===================>]   9.03M  17.8MB/s    in 0.5s    

2020-10-15 21:06:16 (17.8 MB/s) - ‘images_background.zip’ saved [946

In [1]:
!wget https://github.com/manmeet3/Deep_Learning2/raw/master/Asg4/images_evaluation.zip

--2020-10-15 21:06:14--  https://github.com/manmeet3/Deep_Learning2/raw/master/Asg4/images_evaluation.zip
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/manmeet3/Deep_Learning2/master/Asg4/images_evaluation.zip [following]
--2020-10-15 21:06:14--  https://raw.githubusercontent.com/manmeet3/Deep_Learning2/master/Asg4/images_evaluation.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6462886 (6.2M) [application/zip]
Saving to: ‘images_evaluation.zip’

images_evaluation.z 100%[===================>]   6.16M  14.7MB/s    in 0.4s    

2020-10-15 21:06:15 (14.7 MB/s) - ‘images_evaluation.zip’ saved [646

In [5]:
!unzip -q images_background.zip
!unzip -q images_evaluation.zip

In [6]:
!ls

images_background      images_evaluation      sample_data
images_background.zip  images_evaluation.zip


In [11]:
import sys
import numpy as np
import pandas as pd
from imageio import imread # from scipy.misc import imread
import pickle
import os
import matplotlib.pyplot as plt
%matplotlib inline

import cv2
import time

import tensorflow as tf
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model

from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform

from keras.engine.topology import Layer
from keras.regularizers import l2
from keras import backend as K

from sklearn.utils import shuffle

In [12]:
!mkdir data

In [13]:
train_folder = "images_background/"
val_folder = "images_evaluation/"
save_path = "data/"

In [ ]:
def loadimages(path, n=0):
  '''
  Parameters:
  string:path -> Path of train or test directory

  Return Value:
  tuple:(x, y, language_dictionary)
  '''
  X=[]
  y=[]
  cat_dict={}
  lang_dict={}
  curr_y=next

  for alphabet in os.listdir(path):
    print("loading alphabet: " + alphabet)
    lang_dict[alphabet] = [curr_y, None]
    alphabet_path = os.path.join(path, alphabet)


